In [78]:
Pkg.add("Combinatorics")

INFO: Cloning cache of Combinatorics from https://github.com/JuliaMath/Combinatorics.jl.git
INFO: Cloning cache of Polynomials from https://github.com/Keno/Polynomials.jl.git
INFO: Installing Combinatorics v0.3.2
INFO: Installing Polynomials v0.1.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Combinatorics
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [79]:
using Combinatorics

INFO: Precompiling module Polynomials.
INFO: Recompiling stale cache file /home/mrg/.julia/lib/v0.5/Iterators.ji for module Iterators.


In [1]:
using PyPlot

In [100]:
#Starting Grid
NGridSize = 12
const α = .5 #Coeficiente de clon de presa
const β = .5 #Coeficiente de clon de depredador
const ϵ_inicial = -3 #Energia inicial para cada clon de depredador
const Δϵ = 1  #Perdida de energia por paso de tiempo
const γ = .1 #Porcentaje a hacer update en cada turno
const ρ_1 = .333 #Densidad de presas
const ρ_2 = .333 #Densidad de depredadores

Grid = rand(-1:1:1,NGridSize,NGridSize)
GlobalTurnTimer = 0
Espacio_Coordenadas = [(x,y) for x=1:NGridSize, y=1:NGridSize]

12×12 Array{Tuple{Int64,Int64},2}:
 (1,1)   (1,2)   (1,3)   (1,4)   (1,5)   …  (1,9)   (1,10)   (1,11)   (1,12) 
 (2,1)   (2,2)   (2,3)   (2,4)   (2,5)      (2,9)   (2,10)   (2,11)   (2,12) 
 (3,1)   (3,2)   (3,3)   (3,4)   (3,5)      (3,9)   (3,10)   (3,11)   (3,12) 
 (4,1)   (4,2)   (4,3)   (4,4)   (4,5)      (4,9)   (4,10)   (4,11)   (4,12) 
 (5,1)   (5,2)   (5,3)   (5,4)   (5,5)      (5,9)   (5,10)   (5,11)   (5,12) 
 (6,1)   (6,2)   (6,3)   (6,4)   (6,5)   …  (6,9)   (6,10)   (6,11)   (6,12) 
 (7,1)   (7,2)   (7,3)   (7,4)   (7,5)      (7,9)   (7,10)   (7,11)   (7,12) 
 (8,1)   (8,2)   (8,3)   (8,4)   (8,5)      (8,9)   (8,10)   (8,11)   (8,12) 
 (9,1)   (9,2)   (9,3)   (9,4)   (9,5)      (9,9)   (9,10)   (9,11)   (9,12) 
 (10,1)  (10,2)  (10,3)  (10,4)  (10,5)     (10,9)  (10,10)  (10,11)  (10,12)
 (11,1)  (11,2)  (11,3)  (11,4)  (11,5)  …  (11,9)  (11,10)  (11,11)  (11,12)
 (12,1)  (12,2)  (12,3)  (12,4)  (12,5)     (12,9)  (12,10)  (12,11)  (12,12)

In [3]:

YSouth = mod(Row,NGridSize)+1
XWest = mod(Col,  NGridSize) +1
XEast = mod(Col-2,  NGridSize) +1
YNorth = mod(Row-2, NGridSize) +1

2

In [42]:
"""
Esta función revisa si en un vecindario de las 8 casillas adyacentes a una entrada en la matriz de Grid
existe al menos un solo vecino vacío. El código toma en cuenta los casos donde la frontera es toroidal.
Ejemplo:
```julia
julia> a=eye[2,2]
julia> check_vecinos_vacios(1,1)
1
julia> check_vecinos_vacios(1,2)
0
"""
function check_vecinos_vacios(Row::Int,Col::Int)
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    @show Grid[YNorth, Col]== 0 ||Grid[Row, XEast]==0 ||
    Grid[Row, XWest]==0 || Grid[YSouth, Col]==0 ||
    Grid[YNorth,XEast]==0 || Grid[YNorth,XWest]==0 ||
    Grid[YSouth,XEast]==0 || Grid[YSouth,XWest]== 0 return 1
    
    return 0
end

check_vecinos_vacios

In [70]:
Grid = [0 for x=1:5, y=1:5]
Grid[2,2] = -3
Grid[2,1] = 1
Grid

5×5 Array{Int64,2}:
 0   0  0  0  0
 1  -3  0  0  0
 0   0  0  0  0
 0   0  0  0  0
 0   0  0  0  0

In [73]:
moviendo_depredador(2,2,Grid,.5)
#Grid = rand(-1:1:1,NGridSize,NGridSize)
Grid

5×5 Array{Int64,2}:
  0   0  0  0  0
 -3   0  0  0  0
  0  -2  0  0  0
  0   0  0  0  0
  0   0  0  0  0

In [3]:
"""
Esta función revisa si en un vecindario de las 8 casillas aledañas a la entrada (Row,Col) existe al menos
una presa, y regresa un 1 en dado caso, 0 c.o.c. El código toma en cuenta el caso donde la frontera es toroidal.
julia> a=eye[2,2]
julia> check_comer_depredador(1,1)
1
julia> b = zeros[2,2]
julia> check_comer_depredador[1,1]
0
"""
function check_comer_depredador(Row::Int,Col::Int,Grid::Array{Int,2})
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    @show if Grid[YNorth, Col] > 0 || Grid[Row, XEast] > 0 || Grid[Row, XWest] > 0 ||  Grid[YSouth,Col] > 0 || Grid[YNorth,XEast] > 0 || Grid[YNorth,XWest] > 0 || Grid[YSouth,XEast] > 0 ||  Grid[YSouth,XWest] > 0
        return 1
    else return 0
    end
end

check_comer_depredador

In [4]:
"""
Esta función revisa si en un vecindario de las 8 casillas aledañas a la entrada (Row,Col) existe al menos
un movimiento posible para el depredador, y regresa un 1 en dado caso, 0 c.o.c. El código toma en cuenta el caso donde la frontera es toroidal.
julia> a=eye[2,2]
julia> check_mover_depredador(1,1)
1
julia> b = zeros[2,2]
julia> check_comer_depredador[1,1]
0
"""
function check_mover_depredador(Row::Int,Col::Int,Grid::Array{Int,2})
    YSouth = mod(Row,NGridSize)+1
    XWest = mod(Col,  NGridSize) +1
    XEast = mod(Col-2,  NGridSize) +1
    YNorth = mod(Row-2, NGridSize) +1
    
    @show if Grid[YNorth, Col] >= 0 || Grid[Row, XEast] >= 0 || Grid[Row, XWest] >= 0 ||  Grid[YSouth,Col] >= 0 || Grid[YNorth,XEast] >= 0 || Grid[YNorth,XWest] >= 0 || Grid[YSouth,XEast] >= 0 || Grid[YSouth,XWest] >= 0
        return 1
    else return 0
        
    end
end

check_mover_depredador

In [174]:
#Movimiento de presas
function moviendo_presa(Row::Int,Col::Int,α::Float64)
    
    if Grid[Row,Col] <= 0
        return 0 
    end
    
    if check_vecinos_vacios(Row,Col) == 1
        println("moviendo presa")
        
        #Coeficiente de clonado
        rand() < α ? clon = 1 : clon = 0

        YSouth = mod(Row,NGridSize)+1
        XWest = mod(Col,  NGridSize) +1
        XEast = mod(Col-2,  NGridSize) +1
        YNorth = mod(Row-2, NGridSize) +1
        
        while true
        @show behave = rand(1:1:8)
         if behave == 1 && Grid[YNorth,XWest] == 0
            Grid[YNorth,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 2 && Grid[YNorth,Col] == 0
            Grid[YNorth,Col] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 3 && Grid[YNorth,XEast] == 0
            Grid[YNorth,XEast] = 1 
            Grid[Row,Col] = clon
                break
        elseif behave == 4 && Grid[Row,XWest] == 0
            Grid[Row,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 5 && Grid[Row,XEast] == 0
            Grid[Row,XEast] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 6 && Grid[YSouth,XWest] == 0
            Grid[YSouth,XWest] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 7 && Grid[YSouth,Col] == 0
            Grid[YSouth,Col] = 1
            Grid[Row,Col] = clon
                break
        elseif behave == 8 && Grid[YSouth,XEast] == 0
            Grid[YSouth,XEast] = clon
                Grid[Row,Col] = clon
                break
            
            end
        end
    end
end

            


moviendo_presa (generic function with 1 method)

In [64]:
#Movimiento de depredadores
function moviendo_depredador(Row::Int,Col::Int,Grid::Array{Int,2},β::Float64)
    
    if Grid[Row,Col] >= 0
        return 0
    end
    
    puede_moverse = 0
    puede_comer = 0
    
    @show puede_moverse = check_mover_depredador(Row,Col,Grid)
    @show puede_comer   = check_comer_depredador(Row,Col,Grid)
        
    @show puede_comer == 1 ? ϵ_depredador = ϵ_inicial : ϵ_depredador = Grid[Row,Col] + Δϵ 
    
    if check_mover_depredador(Row,Col,Grid) == 1
        
        #Coeficiente de clonado
        rand() < β ? clon = ϵ_inicial : clon = 0
        
        YSouth = mod(Row,NGridSize)+1
        XWest = mod(Col,  NGridSize) +1
        XEast = mod(Col-2,  NGridSize) +1
        YNorth = mod(Row-2, NGridSize) +1
        
        while true
        @show behave = rand(1:1:8)
            if behave == 1 && Grid[YNorth,XWest] >= puede_comer  
                Grid[YNorth,XWest] = ϵ_depredador
                Grid[Row,Col] = clon    
                break
        elseif behave == 2 && Grid[YNorth,Col]  >= puede_comer 
                Grid[YNorth,Col] = ϵ_depredador
                Grid[Row,Col] = clon
                break
        elseif behave == 3 && Grid[YNorth,XEast] >= puede_comer   
                Grid[YNorth,XEast] = ϵ_depredador 
                Grid[Row,Col] = clon
                break
        elseif behave == 4 && Grid[Row,XWest] >= puede_comer   
                Grid[Row,XWest] = ϵ_depredador
                Grid[Row,Col] = clon
                break
        elseif behave == 5 && Grid[Row,XEast] >= puede_comer   
                Grid[Row,XEast] = ϵ_depredador
                Grid[Row,Col] = clon
                break
        elseif behave == 6 && Grid[YSouth,XWest] >= puede_comer 
                Grid[YSouth,XWest] = ϵ_depredador
                Grid[Row,Col] = clon
                break
        elseif behave == 7 && Grid[YSouth,Col] >= puede_comer  
                Grid[YSouth,Col] = ϵ_depredador
                Grid[Row,Col] = clon
                break
        elseif behave == 8 && Grid[YSouth,XEast] >= puede_comer 
                Grid[YSouth,XEast] = ϵ_depredador 
                Grid[Row,Col] = clon
                break
            
        end
        end
    end
end

 

moviendo_depredador (generic function with 1 method)

In [74]:
Grid = eye(5,5)
Grid[2,2] = -3

moviendo_depredador(2,2,Grid,.5)
#Grid = rand(-1:1:1,NGridSize,NGridSize)
Grid

LoadError: LoadError: MethodError: no method matching moviendo_depredador(::Int64, ::Int64, ::Array{Float64,2}, ::Float64)
Closest candidates are:
  moviendo_depredador(::Int64, ::Int64, !Matched::Array{Int64,2}, ::Float64) at In[64]:4
while loading In[74], in expression starting on line 4

In [54]:
#Super Gran Loop
Grid = rand(-1:1:1, NGridSize,NGridSize)
for 1:GlobalTurnTimer=1000
    
    
    

LoadError: LoadError: syntax: incomplete: premature end of input
while loading In[54], in expression starting on line 3

In [60]:
sortperm(index(Grid))


LoadError: LoadError: UndefVarError: index not defined
while loading In[60], in expression starting on line 1

In [64]:
collect(take(Grid,5))

5-element Array{Int64,1}:
  0
  1
  0
  1
 -1

In [69]:
iter=enumerate(Grid)

Enumerate{Array{Int64,2}}([0 0 … 0 0; 1 0 … 1 0; … ; 0 -1 … -1 0; 1 -1 … 0 -1])

In [73]:
count(iter->(iter>0), Grid)

52

In [72]:
count(iter->(iter<0), Grid)

51

In [77]:
Pkg.add("StatsBase")

INFO: No packages to install, update or remove
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of StatsBase
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [78]:
using StatsBase

INFO: Recompiling stale cache file /home/mrg/.julia/lib/v0.5/StatsBase.ji for module StatsBase.


In [114]:
items = [-3,-2,-1,0,1]
weights = [ρ_1 /3, ρ_1 /3, ρ_1 /3, 1-ρ_1-ρ_2, ρ_2]

sample(items,WeightVec(weights),100, replace=false)
AAA = reshape(sample(items,WeightVec(weights),100, replace=true),10,10)

10×10 Array{Int64,2}:
  1  -3   0  -1   0   1  -1  -3  -1   0
 -1  -2   0   1   1   0   0   1  -1   1
 -2  -1   1   0   1  -3  -3   1   1  -3
  0  -2   1   1  -1   1   1  -2  -3   1
  0  -2   0   1  -3   1  -2   1   0  -3
  1  -3   0   0  -3   1   1  -2   0  -3
  0  -1   1   1   0  -1   1   1   1   1
  1   0  -2  -1  -1   1  -1  -1  -1   1
  1   0   1  -2   0   0   1   0   1   1
  0   1   1   1   1   0  -1  -2  -3  -3

In [80]:
collect(permutations([1:NGridSize]))

1-element Array{Array{UnitRange{Int64},1},1}:
 UnitRange{Int64}[1:12]

In [82]:
for i in eachindex(Grid)
    Row,Col = i
    print(Row,Col)
end

LoadError: LoadError: BoundsError
while loading In[82], in expression starting on line 1

In [97]:
for i in CartesianRange(size(Grid))
    println(i, " ", Grid[i])
    [Row,Col] = 
       end

LoadError: LoadError: syntax: unexpected end
while loading In[97], in expression starting on line 4

In [113]:
a1 = [(x,y) for x=1:NGridSize, y=1:NGridSize]

12×12 Array{Tuple{Int64,Int64},2}:
 (1,1)   (1,2)   (1,3)   (1,4)   (1,5)   …  (1,9)   (1,10)   (1,11)   (1,12) 
 (2,1)   (2,2)   (2,3)   (2,4)   (2,5)      (2,9)   (2,10)   (2,11)   (2,12) 
 (3,1)   (3,2)   (3,3)   (3,4)   (3,5)      (3,9)   (3,10)   (3,11)   (3,12) 
 (4,1)   (4,2)   (4,3)   (4,4)   (4,5)      (4,9)   (4,10)   (4,11)   (4,12) 
 (5,1)   (5,2)   (5,3)   (5,4)   (5,5)      (5,9)   (5,10)   (5,11)   (5,12) 
 (6,1)   (6,2)   (6,3)   (6,4)   (6,5)   …  (6,9)   (6,10)   (6,11)   (6,12) 
 (7,1)   (7,2)   (7,3)   (7,4)   (7,5)      (7,9)   (7,10)   (7,11)   (7,12) 
 (8,1)   (8,2)   (8,3)   (8,4)   (8,5)      (8,9)   (8,10)   (8,11)   (8,12) 
 (9,1)   (9,2)   (9,3)   (9,4)   (9,5)      (9,9)   (9,10)   (9,11)   (9,12) 
 (10,1)  (10,2)  (10,3)  (10,4)  (10,5)     (10,9)  (10,10)  (10,11)  (10,12)
 (11,1)  (11,2)  (11,3)  (11,4)  (11,5)  …  (11,9)  (11,10)  (11,11)  (11,12)
 (12,1)  (12,2)  (12,3)  (12,4)  (12,5)     (12,9)  (12,10)  (12,11)  (12,12)

In [115]:
x1,y1=a1[3,2]

(3,2)

In [107]:
Espacio_Coordenadas = reshape(1:NGridSize,NGridSize,NGridSize)

LoadError: LoadError: DimensionMismatch("parent has 12 elements, which is incompatible with size (12,12)")
while loading In[107], in expression starting on line 1

In [128]:
function sampleo_coordenadas(Espacio_Coordenadas,γ,Grid,Row,Col)
    
    sampleo = Array{Int}[]
    for x in 1:NGridSize, y in 1:NGridSize
        
        if γ < rand() && Grid[Row,Col] != 0
                push!([x,y],sampleo)
        end
        
     return sampleo
    end
end

sampleo_coordenadas (generic function with 2 methods)

In [133]:
sampleo = sampleo_coordenadas(Espacio_Coordenadas,.5,Grid,Row,Col)

0-element Array{Array{Int64,N},1}

In [101]:

a = [5,6]

2-element Array{Int64,1}:
 5
 6